# LocalCart scenario part 3b: Analyze static clickstreams

In this notebook, you'll analyze clickstream data to understand how users interact with your online store. 

<img src="https://raw.githubusercontent.com/wdp-beta/get-started/master/notebooks/images/nb3b_static_analysis.png"></img>

A clickstream represents the sequential events that users triggered while they were interacting with the online store. These events include, but are not limited to: 
* view an item
* add an item to cart
* checkout a cart
* complete the checkout process
* abandon the checkout process

This notebook runs on Python 2 with Spark 2.0.

Before you run this notebook, complete the setup tasks https://apsportal.ibm.com/docs/content/getting-started/wdp-beta-scenario.html and run these notebooks:

1. LocalCart scenario part 1: [Generating a Kafka producer (JSON) into MessageHub](https://github.com/wdp-beta/get-started/blob/master/notebooks/localcart-scenario-part-1.ipynb)
1. LocalCart scenario part 2: [Building the streaming pipeline](https://github.com/wdp-beta/get-started/blob/master/notebooks/localcart-scenario-part-2.ipynb)
1. LocalCart scenario part 3: [Analyze customer demographics and sales data](https://github.com/wdp-beta/get-started/blob/master/notebooks/localcart-scenario-part-3.ipynb)

## Table of contents
[Setup](#Setup)<br>
[Load static clickstream data](#Load-static-clickstream-data)<br>
[Re-establish the original clickstream event sequence](#Re-establish-the-original-clickstream-event-sequence)<br>
[Create customer clickstream histories](#Create-customer-clickstream-histories)<br>
[View event statistics](#View-event-statistics)<br>
[View event streams](#View-event-streams)<br>
[Summary and next steps](#Summary-and-next-steps)

## Setup 

Import the libraries that you'll use in this notebook:

In [ ]:
import pixiedust
import pyspark.sql.functions as func
import pyspark.sql.types as types
import re
from datetime import datetime
from decimal import Decimal
from pyspark.sql import SparkSession

## Load static clickstream data
You'll access the clickstream data files in your Object Storage service and then consolidate the data from the many clickstream files into DataFrames by event type. 

You'll create DataFrames for these types of events:
- Login 
- Browsing
- Checkout
- Add an item to a cart
- Logout after purchasing
- Logout without purchasing

### Configure Object Storage credentials

The clickstream files are stored in the Object Storage service that's associated with your DSX account. To access your object storage programmatically, you need to copy in your credentials, which you can see in your Object Storage service details in Bluemix.

1. Open your <a target="_blank" href="https://apsportal.ibm.com/settings/services?context=analytics">Bluemix Data Services list</a>. A list of your provisioned services is displayed.
1. Locate the pre-provisioned **Object Storage** service and click on the service instance name.
1. Open the _Service Credentials_ tab and view the credentials.
1. Copy the credentials text between the curly braces and paste it over the text between the braces in the next cell.
1. Replace `<OBJECT_STORAGE_CONTAINER_NAME>` with the name of your object storage container that you used for your streaming pipelines. Most likely, it's the same container that you're using in this project. You can see the container name for your project on the project **Settings** page, in the **Storage** section.
1. Run the cell.

In [ ]:
# @hidden_cell
# TODO replace with your Object Storage credentials from Bluemix
OS_credentials = {
  "auth_url": "https://identity.open.softlayer.com",
  "projectId": "<...>",
  "region": "<...>",
  "userId": "<...>",
  "password": "<...>"
}

# TODO: replace with your Object Storage container name
container = '<OBJECT_STORAGE_CONTAINER_NAME>'

Define the following function to access your object storage:

In [ ]:
# This function is used to set up the access of Spark to your Object Storage. It references the variables you set up for your credentials.
def set_hadoop_config_with_credentials(credentials, name):
    """This function sets the Hadoop configuration so it is possible to
    access data from Bluemix Object Storage using Spark"""

    prefix = 'fs.swift.service.' + name
    hconf = sc._jsc.hadoopConfiguration()
    hconf.set(prefix + '.auth.url', '{}/v3/auth/tokens'.format(credentials['auth_url']))
    hconf.set(prefix + '.tenant', credentials['projectId']) 
    hconf.set(prefix + '.username', credentials['userId'])
    hconf.set(prefix + '.password', credentials['password'])
    hconf.set(prefix + '.region', credentials['region'])
    hconf.setInt(prefix + '.http.port', 8080)
    hconf.setBoolean(prefix + '.public', False)
    hconf.set(prefix + '.auth.endpoint.prefix', 'endpoints') # optional

name = 'keystone'
set_hadoop_config_with_credentials(OS_credentials, name)

spark = SparkSession.builder.getOrCreate()

### Load login clickstream events 
Consolidate login data from multiple clickstream files into a DataFrame named `login_df`:

In [ ]:
# load login events
filename_pattern = 'login_*.csv'

path = 'swift://{}.{}/{}'.format(container, name, filename_pattern)

login_schema = types.StructType([
                                 types.StructField('customer_id', types.StringType(), True),
                                 types.StructField('click_event_type', types.StringType(), True),
                                 types.StructField('total_price_of_basket', types.DecimalType(10,2), True),
                                 types.StructField('total_number_of_items_in_basket', types.IntegerType(), True),
                                 types.StructField('total_number_of_distinct_items_in_basket', types.IntegerType(), True),
                                 types.StructField('click_event_time', types.TimestampType(), True),         
                                ])

print 'Loading clickstream data for login events from {} ...'.format(path)

login_df = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'false')\
  .option('timestampFormat', 'yyyy-MM-dd HH:mm:ss z')\
  .load(path, schema = login_schema)

if login_df:
    print 'Data structure: {}'.format(login_df)
    print 'Event count: {:,}'.format(login_df.count())
    print 'Example event: {}'.format(login_df.head())
else:
    print 'Fatal error loading login events.'

### Load browsing clickstream events 
Consolidate browsing data from multiple clickstream files into a DataFrame named `browsing_df`:

In [ ]:
# load browsing events
filename_pattern = 'browsing_*.csv'

path = 'swift://{}.{}/{}'.format(container, name, filename_pattern)

browsing_schema = types.StructType([
                                    types.StructField('customer_id', types.StringType(), True),
                                    types.StructField('click_event_type', types.StringType(), True),
                                    types.StructField('total_price_of_basket', types.DecimalType(10,2), True),
                                    types.StructField('total_number_of_items_in_basket', types.IntegerType(), True),
                                    types.StructField('total_number_of_distinct_items_in_basket', types.IntegerType(), True),
                                    types.StructField('click_event_time', types.TimestampType(), True),            
                                   ])

print 'Loading clickstream data for browsing events from {} ...'.format(path)

browsing_df = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'false')\
  .option('timestampFormat', 'yyyy-MM-dd HH:mm:ss z')\
  .load(path, schema = browsing_schema)

if browsing_df:
    print 'Data structure: {}'.format(browsing_df)
    print 'Event count: {:,}'.format(browsing_df.count())
    print 'Example event: {}'.format(browsing_df.head())
else:
    print 'Fatal error loading browsing events.'

### Load checkout clickstream events 
Consolidate checkout data from multiple clickstream files into a DataFrame named `checkout_df`:

In [ ]:
# load checkout events
filename_pattern = 'checkout_*.csv'

path = 'swift://{}.{}/{}'.format(container, name, filename_pattern)

checkout_schema = types.StructType([
                                    types.StructField('customer_id', types.StringType(), True),
                                    types.StructField('click_event_type', types.StringType(), True),
                                    types.StructField('total_price_of_basket', types.DecimalType(10,2), True),
                                    types.StructField('total_number_of_items_in_basket', types.IntegerType(), True),
                                    types.StructField('total_number_of_distinct_items_in_basket', types.IntegerType(), True),
                                    types.StructField('session_duration', types.IntegerType(), True),
                                    types.StructField('click_event_time', types.TimestampType(), True),            
                                   ])

print 'Loading clickstream data for checkout events from {} ...'.format(path)

checkout_df = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'false')\
  .option('timestampFormat', 'yyyy-MM-dd HH:mm:ss z')\
  .load(path, schema = checkout_schema)

if checkout_df:
    print 'Data structure: {}'.format(checkout_df)
    print 'Event count: {:,}'.format(checkout_df.count())
    print 'Example event: {}'.format(checkout_df.head())
else:
    print 'Fatal error loading checkout events.'

### Load add_to_cart clickstream events 
Consolidate data for adding items to a cart from multiple clickstream files into a DataFrame named `add_to_cart_df`:

In [ ]:
# load add_to_cart events
filename_pattern = 'addtocart_*.csv'

path = 'swift://{}.{}/{}'.format(container, name, filename_pattern)

add_to_cart_schema = types.StructType([
                                       types.StructField('customer_id', types.StringType(), True),
                                       types.StructField('click_event_type', types.StringType(), True),
                                       types.StructField('product_name', types.StringType(), True),
                                       types.StructField('product_category', types.StringType(), True),
                                       types.StructField('product_price', types.DecimalType(10,2), True),
                                       types.StructField('total_price_of_basket', types.DecimalType(10,2), True),
                                       types.StructField('total_number_of_items_in_basket', types.IntegerType(), True),
                                       types.StructField('total_number_of_distinct_items_in_basket', types.IntegerType(), True),
                                       types.StructField('click_event_time', types.TimestampType(), True),            
                                      ])

print 'Loading clickstream data for add_to_cart events from {} ...'.format(path)

add_to_cart_df = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'false')\
  .option('timestampFormat', 'yyyy-MM-dd HH:mm:ss z')\
  .load(path, schema = add_to_cart_schema)

if add_to_cart_df:
    print 'Data structure: {}'.format(add_to_cart_df)
    print 'Event count: {:,}'.format(add_to_cart_df.count())
    print 'Example event: {}'.format(add_to_cart_df.head())
else:
    print 'Fatal error loading add_to_cart events.'

### Load logout_with_purchase clickstream events 
Consolidate data for logging out after purchasing from multiple clickstream files into a DataFrame named `logout_with_purchase_df`:

In [ ]:
# load logout_with_purchase events
filename_pattern = 'logoutwithpurchase_*.csv'

path = 'swift://{}.{}/{}'.format(container, name, filename_pattern)

logout_with_purchase_schema = types.StructType([
                                                types.StructField('customer_id', types.StringType(), True),
                                                types.StructField('click_event_type', types.StringType(), True),
                                                types.StructField('total_price_of_basket', types.DecimalType(10,2), True),
                                                types.StructField('total_number_of_items_in_basket', types.IntegerType(), True),
                                                types.StructField('total_number_of_distinct_items_in_basket', types.IntegerType(), True),
                                                types.StructField('session_duration', types.IntegerType(), True),                    
                                                types.StructField('click_event_time', types.TimestampType(), True),            
                                               ])

print 'Loading clickstream data for logout_with_purchase events from {} ...'.format(path)

logout_with_purchase_df = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'false')\
  .option('timestampFormat', 'yyyy-MM-dd HH:mm:ss z')\
  .load(path, schema = logout_with_purchase_schema)

if logout_with_purchase_df:
    print 'Data structure: {}'.format(logout_with_purchase_df)
    print 'Event count: {:,}'.format(logout_with_purchase_df.count())
    print 'Example event: {}'.format(logout_with_purchase_df.head())
else:
    print 'Fatal error loading logout_with_purchase events.'

### Load logout_without_purchase clickstream events 
Consolidate data for logging out without purchasing from multiple clickstream files into a DataFrame named `logout_without_purchase_df`:

In [ ]:
# load logout_without_purchase events
filename_pattern = 'logoutwithoutpurchase_*.csv'

path = 'swift://{}.{}/{}'.format(container, name, filename_pattern)

logout_without_purchase_schema = types.StructType([
                                                   types.StructField('customer_id', types.StringType(), True),
                                                   types.StructField('click_event_type', types.StringType(), True),
                                                   types.StructField('total_price_of_basket', types.DecimalType(10,2), True),
                                                   types.StructField('total_number_of_items_in_basket', types.IntegerType(), True),
                                                   types.StructField('total_number_of_distinct_items_in_basket', types.IntegerType(), True),
                                                   types.StructField('session_duration', types.IntegerType(), True),                    
                                                   types.StructField('click_event_time', types.TimestampType(), True),            
                                                  ])

print 'Loading clickstream data for logout_without_purchase events from {} ...'.format(path)

logout_without_purchase_df = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'false')\
  .option('timestampFormat', 'yyyy-MM-dd HH:mm:ss z')\
  .load(path, schema = logout_without_purchase_schema)

if logout_without_purchase_df:
    print 'Data structure: {}'.format(logout_without_purchase_df)
    print 'Event count: {:,}'.format(logout_without_purchase_df.count())
    print 'Example event: {}'.format(logout_without_purchase_df.head())
else:
    print 'Fatal error loading logout_without_purchase events.'

## Re-establish the original clickstream event sequence

Now create a DataFrame that consolidates all the clickstream event data in the right order:

In [ ]:
# Union click events
events_df = (login_df.select("click_event_time", "customer_id", "click_event_type").
                    union(browsing_df.select("click_event_time", "customer_id", "click_event_type")).
                    union(add_to_cart_df.select("click_event_time", "customer_id", "click_event_type")).
                    union(checkout_df.select("click_event_time", "customer_id", "click_event_type")).                             
                    union(logout_with_purchase_df.select("click_event_time", "customer_id", "click_event_type")).  
                    union(logout_without_purchase_df.select("click_event_time", "customer_id", "click_event_type")))

if events_df:
    print 'Data structure: {}'.format(events_df)
    print 'Event count: {:,}'.format(events_df.count())
else:
    print 'Fatal error unioning clickstream events.'

Print the count of each type of event:

In [ ]:
events_df.groupBy("click_event_type").count().orderBy('count', ascending = False).collect()

## Create customer clickstream histories 
First create single character abbreviations for each type of event. Then create a DataFrame with event histories for each customer.

In [ ]:
event_type_dict = {
    'login': 'l',
    'browsing': 'b',
    'add_to_cart': 'a',
    'checkout': 'c',
    'logout_with_purchase': 'p',
    'logout_without_purchase': 'n'
}

# create clickstream history for each customer: Row(customer_id=u'13602', events=u'bbbbblbbbbblaaacp', CP_count=1, CN_count=0)
# each character in events represents an event from the map above; 'U' indicates an unknown event
customer_cs = {}
for row in events_df.orderBy('click_event_time').collect():
    if customer_cs.get(row.customer_id):
        customer_cs[row.customer_id] += event_type_dict.get(row.click_event_type,'U')
    else:
        customer_cs[row.customer_id] = event_type_dict.get(row.click_event_type,'U')

# create dataframe
events_history_df = sc.parallelize([(k,)+(v,) for k,v in customer_cs.items()]).toDF(['customer_id','events'])

def matchPattern(col, pattern):
    return len(re.findall(pattern,col))

matchPatternUDF = func.udf(lambda c,p: matchPattern(c,p), types.IntegerType())
# count number of product views
events_history_df = events_history_df.withColumn("B_count", matchPatternUDF(events_history_df['events'],func.lit('b')))
# count number of add-to-cart
events_history_df = events_history_df.withColumn("BA_count", matchPatternUDF(events_history_df['events'],func.lit('ba')))
# count number of completed purchases (checkout -> logout_with_purchase)
events_history_df = events_history_df.withColumn("CP_count", matchPatternUDF(events_history_df['events'],func.lit('cp')))
# count number of aborted checkouts (checkout -> logout_without_purchase)
events_history_df = events_history_df.withColumn("CN_count", matchPatternUDF(events_history_df['events'],func.lit('cn')))

## View event statistics
Look at the summary statistics about the number of customers and each type of event.

In [ ]:
# Number of customers
print 'Customer count: {:,}'.format(events_history_df.count())
# Number of product views
print 'Product views: {:,}'.format(events_history_df.agg({'B_count': 'sum'}).collect()[0][0])
# Number of add-to-cart events
print 'Cart additions: {:,}'.format(events_history_df.agg({'BA_count': 'sum'}).collect()[0][0])
# Number of customers that have completed at least one purchase
print 'Customers with purchase: {:,}'.format(events_history_df.filter('CP_count > 0').count())
# Number of customers that have completed more than one purchase
print 'Customers with multiple purchases: {:,}'.format(events_history_df.filter('CP_count > 1').count())
# Number of abandoned checkouts (checkout was never completed)
print 'Abandoned carts: {:,}'.format(events_history_df.agg({'CN_count': 'sum'}).collect()[0][0])

## View event streams

Now take a look at some customer event streams. 

Remember the event type abbreviations:
- l = login
- b = browsing
- a = add to cart
- c = checkout
- p = logout after purchasing
- n = logout without purchasing
- U = unknown event

In [ ]:
# sample event streams for 10 customers
events_history_df.head(10) 

## Summary and next steps
You've learned how to get the data from multiple event streams into a format that you can use to analyze customer behavior.

Next, learn how to create a dashboard for streaming data in [LocalCart scenario part 4: Visualize streaming data in a real-time dashboard](https://github.com/wdp-beta/get-started/blob/master/notebooks/localcart-scenario-part-4.ipynb) notebook.


## Author

Patrick Titzler is a customer advocate for Watson Data Platform at IBM.
<hr>
Copyright &copy; IBM Corp. 2017. This notebook and its source code are released under the terms of the MIT License.